# Setup, Libraries
Download sample images and a label typeface for this lab, by pasting and running the following command in a new cell. Each line begins with an exclamation point, which tells the kernel to run these bash commands within the notebook cells.

In [1]:
!mkdir sample_images
!wget https://storage.googleapis.com/generativeai-downloads/images/socks.jpg -O sample_images/Socks.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/vegetables.jpg -O sample_images/Vegetables.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/Japanese_Bento.png -O sample_images/Japanese_bento.png -q
!wget https://storage.googleapis.com/generativeai-downloads/images/Cupcakes.jpg -O sample_images/Cupcakes.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/origamis.jpg -O sample_images/Origamis.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/fruits.jpg -O sample_images/Fruits.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/cat.jpg -O sample_images/Cat.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/pumpkins.jpg -O sample_images/Pumpkins.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/breakfast.jpg -O sample_images/Breakfast.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/bookshelf.jpg -O sample_images/Bookshelf.jpg -q
!wget https://storage.googleapis.com/generativeai-downloads/images/spill.jpg -O sample_images/Spill.jpg -q
!sudo apt-get install fonts-noto-cjk  # For Noto Sans CJK JP

mkdir: cannot create directory ‘sample_images’: File exists
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20201206-cjk+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [2]:
from PIL import Image

import io
import os
import requests
from io import BytesIO
import IPython
from IPython.display import Markdown, Video, display

from google import genai
from google.genai import types

from gemini_for_cv_utils import plot_bounding_boxes, parse_json, \
        parse_segmentation_masks, plot_segmentation_masks, generate_point_html, \
        get_url_from_gcs

In [3]:
os.environ

environ{'SHELL': '/bin/bash',
        'CONDA_EXE': '/opt/conda/bin/conda',
        '_CE_M': '',
        'VERTEX_PRODUCT': 'WORKBENCH_INSTANCE',
        'GRPC_FORK_SUPPORT_ENABLED': '0',
        'DL_ANACONDA_HOME': '/opt/conda',
        'FRAMEWORK_FILE_PATH': '/opt/deeplearning/metadata/framework',
        'GOOGLE_CLOUD_PROJECT': 'qwiklabs-gcp-04-27a9e1d9ae22',
        'POST_STARTUP_SCRIPT_PATH': '/opt/c2d/post_start.sh',
        'DL_PATH_DEPS': '/opt/deeplearning/deps',
        'DL_BIN_PATH': '/opt/deeplearning/bin',
        'XML_CATALOG_FILES': 'file:///opt/conda/etc/xml/catalog file:///etc/xml/catalog',
        'KERNEL_LAUNCH_TIMEOUT': '598',
        'BINARIES_PATH': '/opt/deeplearning/binaries',
        'PWD': '/home/jupyter',
        'LOGNAME': 'jupyter',
        'CONDA_PREFIX': '/opt/conda',
        'JPY_SESSION_NAME': '/home/jupyter/gemini_image_understanding.ipynb',
        'TENSORBOARD_PROXY_URL': '/proxy/%PORT%/',
        'GOOGLE_CLOUD_REGION': 'us-central1',
        'PACKAGE_

In [4]:
project_id, = !gcloud config get project
print(project_id)

client = genai.Client(vertexai=True,
                    project=project_id,
                    location="us-central1")

model_name = "gemini-2.5-flash"

qwiklabs-gcp-04-27a9e1d9ae22


In [5]:
# set safety_settings to BLOCK_ONLY_HIGH dangerous content, minimizing the potential number of false alarms for content safety
bounding_box_system_instructions = """
    Return bounding boxes as a JSON array with labels.
    Never return masks or code fencing. Limit to 25 objects.
    If an object is present multiple times, name them according
    to their unique characteristic (colors, size, position,
    unique characteristics, etc.).
    """

# Minimize safety-related errors
safety_settings = [
    types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_ONLY_HIGH",
    ),
]

# Task 6. Use Gemini to extract insights from video

In this task, you will analyze a clip of video from the first televised US presidential debate between John F. Kennedy and Richard Nixon. This debate is largely believed to have been critical in securing JFK's victory due to his calm and personable demeanor for the broadcast.


In [8]:
video_uri = "gs://qwiklabs-gcp-04-27a9e1d9ae22-bucket/nixon_kennedy_debate_clip.mp4"

# Display the video in the notebook
display(Video(get_url_from_gcs(video_uri)))

In [9]:
video_analysis_prompt = """
    You are an expert in politics and history.
    Make observations beginning with video timecode that compare and contrast
    each candidate's points as well as their apparent mood, composure, and agitation.
"""

text_and_video_content = [
    types.UserContent(
        parts=[
            types.Part.from_text(text=video_analysis_prompt),
            types.Part.from_uri(
                file_uri= video_uri,
                mime_type= 'video/mp4',
            )
        ]
    )
]

response = client.models.generate_content(
    model=model_name,
    contents=text_and_video_content,
    config = types.GenerateContentConfig(
        temperature=0.5,
        safety_settings=safety_settings,
    )
)

Markdown(response.text)

Here are observations comparing and contrasting each candidate's points, mood, composure, and agitation during this segment, presented with video timecodes:

*   **0:00-0:20 (Kennedy - Listening to Question):** Senator Kennedy listens intently to the question regarding the seriousness of the communist threat within the United States. He appears calm and composed, with a serious but unagitated expression. His gaze is focused, indicating active listening.

*   **0:20-0:50 (Kennedy - Response on Communism):**
    *   **Points:** Kennedy acknowledges the seriousness of the internal communist threat and the need for vigilance, supporting laws, the Department of Justice, and the FBI. However, he quickly pivots to emphasize that the *major threat is external*. This frames the internal threat as manageable if the nation remains strong and united against external adversaries.
    *   **Mood, Composure, Agitation:** Kennedy maintains a calm, measured, and thoughtful demeanor. His tone is steady and confident, with no apparent signs of agitation. He uses subtle head movements for emphasis but avoids any overt gestures that might suggest discomfort or defensiveness. He projects an image of cool, intellectual leadership.

*   **0:51-0:57 (Nixon - Introduction to Response):** Vice President Nixon begins by directly agreeing with Senator Kennedy's "appraisal generally" regarding the internal communist threat. He looks directly into the camera, appearing serious and composed.

*   **0:57-2:27 (Nixon - Response on Communism):**
    *   **Points:** Nixon agrees that communism within the US has been and will continue to be a problem, framing it as part of a global "Cold War" waged by Khrushchev. He emphasizes the need for continued alertness but then adds a crucial caveat: the importance of being *fair* and upholding American freedoms. He suggests that by upholding these freedoms, the US fights communism more effectively than by compromising its values. He differentiates his approach by stating the question is not one of shared "goals" (which they both have, like fighting communism and improving society) but of "means," implying his methods are more effective.
    *   **Mood, Composure, Agitation:** Nixon is highly composed and articulate. His facial expressions are earnest and determined, conveying a strong sense of conviction. He speaks with a firm, steady voice, and his eye contact is direct and unwavering. While passionate about his points, he shows no signs of agitation. He appears confident and in control, using his response to subtly differentiate his approach on civil liberties and the broader Cold War strategy.

*   **2:28-2:53 (Nixon - Listening to Question):** Nixon is now shown standing at a podium, listening to a question that challenges his previous statement about the Eisenhower administration's role in building schools and hospitals, implying these are local responsibilities. He appears attentive and ready to counter the implied criticism.

*   **2:53-4:10 (Nixon - Response on Domestic Progress):**
    *   **Points:** Nixon strongly refutes the idea that progress in areas like schools and hospitals is solely a federal or local matter. He argues that "America moves" when the federal government, state governments, local governments, *and* "individual enterprise" all contribute. He specifically credits the Eisenhower administration's policies with encouraging the private sector, which in turn generated the wealth needed for these public improvements. This highlights a core Republican philosophy of empowering the private sector and local control over large-scale federal intervention.
    *   **Mood, Composure, Agitation:** Nixon is assertive and confident in his delivery. He uses hand gestures to emphasize his points, particularly when listing the different contributing factors to national progress. His tone is firm and declarative, reflecting his conviction in his economic philosophy. He is clearly not agitated but rather energized by the opportunity to articulate his administration's achievements and principles.

*   **4:11-5:09 (Kennedy - Response on Domestic Progress):**
    *   **Points:** Kennedy directly counters Nixon's argument, stating that local school districts are already at a "point of diminishing returns" with property taxes, implying they need federal assistance. He asserts that the US is a rich and powerful nation, but what's needed is *presidential leadership* to set clear goals for the next decade in education, economic growth, and natural resources. He explicitly frames this in terms of competition with the Soviet Union, arguing that the US must be "ahead" by 1970, not just comparing itself to its past. This strongly advocates for a more active and interventionist federal government role, driven by national purpose and international competition.
    *   **Mood, Composure, Agitation:** Kennedy's demeanor shifts slightly here. While still composed, he becomes more animated and passionate. His voice gains a greater sense of urgency, and he uses more emphatic hand gestures (e.g., a chopping motion to emphasize "exactly what we must do"). He projects a sense of vision and a call to action, conveying that the current pace is insufficient to meet global challenges. He is not agitated, but rather displays a controlled intensity and a desire to inspire a more aggressive national effort.

**Overall Comparison & Contrast:**

*   **Points:**
    *   **Communism:** Both agree on the internal threat's seriousness. Kennedy prioritizes the external threat and existing legal structures. Nixon emphasizes upholding American freedoms while combating internal communism and sees the Cold War as a global, multi-front struggle.
    *   **Domestic Progress/Role of Government:** This is the clearest point of contrast. Nixon champions individual enterprise and local/state government as the primary drivers of progress, with the federal government's role being to encourage this. Kennedy argues for robust federal leadership and intervention, citing the limitations of local funding and the urgent need to compete with the Soviet Union on all fronts (education, economy, resources).
*   **Mood, Composure, Agitation:**
    *   **Kennedy:** Generally calm and thoughtful, but capable of shifting to a more urgent and passionate tone when discussing the need for national leadership and global competition. He maintains strong composure throughout, but his intensity fluctuates.
    *   **Nixon:** Consistently composed, earnest, and confident. He articulates his points with a firm, declarative style, showing no signs of agitation but rather a strong, unwavering conviction in his arguments and the Eisenhower administration's record.

In [10]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.74021540338986,
      content=Content(
        parts=[
          Part(
            text="""Here are observations comparing and contrasting each candidate's points, mood, composure, and agitation during this segment, presented with video timecodes:

*   **0:00-0:20 (Kennedy - Listening to Question):** Senator Kennedy listens intently to the question regarding the seriousness of the communist threat within the United States. He appears calm and composed, with a serious but unagitated expression. His gaze is focused, indicating active listening.

*   **0:20-0:50 (Kennedy - Response on Communism):**
    *   **Points:** Kennedy acknowledges the seriousness of the internal communist threat and the need for vigilance, supporting laws, the Department of Justice, and the FBI. However, he quickly pivots to emphasize that the *major threat is external*. This frames the internal th